# Обратная связь




### Почему мы начали с картинок?

> Рассказать про применение сетей не только в компьютерном зрении <...>
Многие студенты <...> не занимаются на прямую компьютерным зрением, а планируют использовать глубинное обучение для анализа табличных данных или, например, данных на графах.

Короткий ответ:
- не используйте DL для табличек

Длинный ответ:
- другие области (тексты, аудио, ряды и немного графы) мы тоже разберем!
- у вас есть подходящая интуиция
- визуализируемые представления
- наиболее развитая область: приемы и инструменты сначала обкатывают здесь, потом переносят куда-то еще
- хорошие датасеты и понятные задачи

- 


### Про предопределенность и основы

> Знакомство с базовыми слоями PyTorch <...> на эти штуки порой отводится несколько полноценных семинаров.
Насколько я понимаю понятно, что структура домашек и тд уже составлена, мой коммент не имеет особого смысла : )


1. Про базовые слои PT можно провести отдельное занятие, но ...

2. Нет структура домашек заранее не фиксирована: у нас есть набор вещей, которые мы хотим рассказать, что-то идет в домашки, что-то в семинар. Мы открыты к предложениям.



### Про непонятки

> Прохожу курс по DL с нуля. Очень много вещей не понял на лекции. Сейчас смотрю базовый курс по DL от вышки, чтобы вникнуть

**Это не ваш недостаток.**

Не стесняйтесь, пишите в общий чат.

Мы можем выделить занятие на подробное объяснение.

# Computer vision part 2

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/m12sl/dl-hse-2021/blob/master/06-computer-vision-2/seminar.ipynb)


- How to get grads w.r.t. inputs?
- Naive Adversarial Attacks
- FGSM
- Let's make creappy images!

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import json

from tqdm.auto import tqdm

with open("imagenet_1k.json") as fin:
    idx2cls = {int(k): v for k, v in json.load(fin).items()}

In [ ]:
# заготовим картинку чтобы была
! wget https://sobakibalabaki.com/wp-content/uploads/2017/06/dog-and-horsejpg.jpg -O doge-and-horsy.jpg

# Предварительная подготовка

Нам потребуется конвертить картинки в тензора и обратно


```
    mean = np.array([0.485, 0.456, 0.406]).reshape(1,1,3)
    std =  np.array([0.229, 0.224, 0.225]).reshape(1,1,3)
```

In [ ]:
def uint2tv(x, channel_dim=-1):
    assert x.shape[channel_dim] == 3, f"Wrong shape"
    return x


def tv2uint(x, channel_dim=-1):
    assert x.shape[channel_dim] == 3, f"Wrong shape"
    return x


def load(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     img = cv2.resize(img, (300, 210))  
    x = uint2tv(img)
    print(x.shape)
    img = np.transpose(x, [2, 0, 1])[None, ...]
    img = torch.tensor(img)
    # [1, 3, w, h]
    return img


tt = load("./doge-and-horsy.jpg")

x = tv2uint(tt.numpy(), channel_dim=1)
print(x.shape)
x = np.transpose(x[0, ...], [1, 2, 0])
plt.imshow(x)

Добавьте код для сохранения интересных нам активаций во время прямого прохода

In [ ]:
class VeryWrapper(nn.Module):
    def __init__(self, model, layer_of_interest=None):
        super().__init__()
        self.model = model        
    def forward(self, x):
        out = self.model(x)
        return out
        
        
net = torchvision.models.resnet50(pretrained=True)
net = net.eval()
model = VeryWrapper(net)

horse = load("./doge-and-horsy.jpg")

y = model(horse)

# Наивная атака градиентным спуском

Давайте оптимизировать входную картинку так, чтобы изменить результат работы сети

In [ ]:
def naive_attack(init_tensor, target, num_iters=50, epsilon=1e-5):
    x = init_tensor.clone().detach()
    x.requires_grad = True
    <todo>
    
        
img = load("./doge-and-horsy.jpg")
naive_attack(img, 0, epsilon=100.0)

Картинка должна остаться картинкой!
Это означает, что изменения должны пережить переход из fp32 -> uin8.

Как этого добиться?

Напишите тесты на преодоление air-gap:

In [ ]:
def uint8_test(original, img, net):
    pass

def jpeg_compression_test(original, img, net):
    pass

# FGSM

Используем вместо градиентов знак градиента

In [ ]:
def fgsm_attack(init_tensor, target, num_iters=50, epsilon=0.01):
    x = init_tensor.clone().detach()
    x.requires_grad = True
    <todo>

        
img = load("./doge-and-horsy.jpg")
fsgm_attack(img, 0, epsilon=0.01)

# Генерируем странные картинки

Что если оптимизировать не CE, а величину активации какого-нибудь слоя или нейрона?

In [ ]:
net

In [ ]:
<todo>